In [30]:
import os

In [41]:
os.path.dirname(sys.executable)

'/opt/conda/bin'

In [32]:
import sys
!{sys.executable} -m pip install selenium

In [42]:
import shutil

shutil.move("geckodriver", "/opt/conda/bin/geckodriver")






'/opt/conda/bin/geckodriver'

In [48]:
from selenium import webdriver

In [51]:
driver = webdriver.Firefox()
driver.get("http://www.python.org")
assert "Python" in driver.title

WebDriverException: Message: Process unexpectedly closed with status 1


In [ ]:
driver.get("https://stromae-90-01z.dev.insee.io/questionnaire/90-01z/unite-enquetee/11")
elem = driver.find_element_by_css_selector('.undefined:nth-child(2) > .MuiButton-label')
elem.get_attribute('innerHTML')